# Assignment #1 Solution

## Step 1: Load and select the Philadelphia data

In [1]:
import pandas as pd

In [2]:
path = "./data/Zip_zhvi_uc_sfrcondo_tier_0.33_0.67_sm_sa_month.csv"
data = pd.read_csv(path)

In [3]:
data.head()

,RegionID,SizeRank,RegionName,RegionType,StateName,State,City,Metro,CountyName,2000-01-31,...,2021-10-31,2021-11-30,2021-12-31,2022-01-31,2022-02-28,2022-03-31,2022-04-30,2022-05-31,2022-06-30,2022-07-31
0,61639,0,10025,Zip,NY,NY,New York,New York-Newark-Jersey City,New York County,329164.0,...,1153364.0,1152736.0,1153314.0,1159292.0,1171216.0,1190200.0,1207107.0,1221417.0,1227148.0,1234232.0
1,84654,1,60657,Zip,IL,IL,Chicago,Chicago-Naperville-Elgin,Cook County,311718.0,...,523727.0,526511.0,528499.0,529879.0,530092.0,532758.0,534840.0,539859.0,543658.0,546709.0
2,61637,2,10023,Zip,NY,NY,New York,New York-Newark-Jersey City,New York County,510209.0,...,1517150.0,1521442.0,1521759.0,1532449.0,1542269.0,1559390.0,1572653.0,1591368.0,1600569.0,1607770.0
3,91982,3,77494,Zip,TX,TX,Katy,Houston-The Woodlands-Sugar Land,Harris County,224385.0,...,428419.0,435498.0,441231.0,447322.0,456848.0,468519.0,482272.0,493709.0,501230.0,503505.0
4,84616,4,60614,Zip,IL,IL,Chicago,Chicago-Naperville-Elgin,Cook County,407594.0,...,669648.0,674293.0,677215.0,679126.0,678889.0,681982.0,684421.0,690214.0,695114.0,698644.0


To select Philadelphia data, we have to select by the `City` column and the `State` column.

In [4]:
selection = (data['City'] == 'Philadelphia') & (data['State'] == 'PA')
data = data.loc[selection]

In [5]:
data.head()

,RegionID,SizeRank,RegionName,RegionType,StateName,State,City,Metro,CountyName,2000-01-31,...,2021-10-31,2021-11-30,2021-12-31,2022-01-31,2022-02-28,2022-03-31,2022-04-30,2022-05-31,2022-06-30,2022-07-31
125,65810,126,19143,Zip,PA,PA,Philadelphia,Philadelphia-Camden-Wilmington,Philadelphia County,60701.0,...,173114.0,172087.0,171445.0,171542.0,171680.0,171878.0,171607.0,171333.0,171771.0,172611.0
247,65779,249,19111,Zip,PA,PA,Philadelphia,Philadelphia-Camden-Wilmington,Philadelphia County,85062.0,...,257911.0,260104.0,262257.0,263715.0,264809.0,265684.0,267222.0,269460.0,272201.0,274446.0
338,65791,340,19124,Zip,PA,PA,Philadelphia,Philadelphia-Camden-Wilmington,Philadelphia County,47155.0,...,156225.0,157780.0,159029.0,159274.0,159886.0,160780.0,161929.0,163625.0,165020.0,166009.0
423,65787,426,19120,Zip,PA,PA,Philadelphia,Philadelphia-Camden-Wilmington,Philadelphia County,59285.0,...,161167.0,161807.0,162634.0,162972.0,163597.0,164008.0,164887.0,165860.0,167321.0,168524.0
509,65772,512,19104,Zip,PA,PA,Philadelphia,Philadelphia-Camden-Wilmington,Philadelphia County,74255.0,...,220270.0,221454.0,222006.0,220760.0,217933.0,216447.0,216424.0,218663.0,220453.0,223443.0


## Step 2: Calculate the annual ZHVI per ZIP code

First, remove unnecessary columns from the data frame.

In [6]:
data = data.drop(
    [
        "RegionID",
        "RegionType",
        "City",
        "StateName",
        "State",
        "Metro",
        "CountyName",
        "SizeRank",
    ],
    axis=1,
)

Next, melt the data frame into long format.

In [7]:
melted = data.melt(id_vars=["RegionName"], value_name='ZHVI', var_name='Date')

In [8]:
melted

,RegionName,Date,ZHVI
0,19143,2000-01-31,60701.0
1,19111,2000-01-31,85062.0
2,19124,2000-01-31,47155.0
3,19120,2000-01-31,59285.0
4,19104,2000-01-31,74255.0
...,...,...,...
12461,19153,2022-07-31,247560.0
12462,19118,2022-07-31,746009.0
12463,19102,2022-07-31,347614.0
12464,19127,2022-07-31,318732.0


Convert the `Date` from a string to a pandas `Datetime` object, and then extract the year from the data and store it as the `Year` column.

In [9]:
melted['Date'] = pd.to_datetime(melted['Date'])
melted['Year'] = melted['Date'].dt.year

To calculate the annual ZHVI: 

- group by ZIP code (`RegionName`) and year (`Year`)
- take the average of the `ZVHI` column for each group

In [10]:
annual_ZHVI = melted.groupby(['RegionName', 'Year'])['ZHVI'].mean()

In [11]:
annual_ZHVI.head()

RegionName  Year
19102       2000    152667.833333
            2001    182936.583333
            2002    217766.666667
            2003    247848.000000
            2004    288225.000000
Name: ZHVI, dtype: float64

Convert from a Series object to a DataFrame by resetting the index.

In [12]:
annual_ZHVI = annual_ZHVI.reset_index()

In [13]:
annual_ZHVI.head()

,RegionName,Year,ZHVI
0,19102,2000,152667.833333
1,19102,2001,182936.583333
2,19102,2002,217766.666667
3,19102,2003,247848.000000
4,19102,2004,288225.000000


## Step 3: Find the ZIP codes with max ZHVI for each year

We need to calculate a quantity for each year, so the first step is to group the data by the `Year` column.

In [14]:
year_grps = annual_ZHVI.groupby('Year')
print(year_grps)

Next, we'll define a function that calculates the ZIP code with the max ZHVI for each year's data.

In [15]:
def get_max_ZHVI_zip(grp):
    
    # this will give you the index of the row that has
    # the max ZHVI value
    idx_max = grp['ZHVI'].idxmax()
    
    # return the RegionName in the row with the max ZVHI
    return grp.loc[idx_max, ["RegionName", "ZHVI"]]

In [16]:
max_ZIPs = year_grps.apply(get_max_ZHVI_zip)

In [17]:
max_ZIPs

,RegionName,ZHVI
Year,,
2000,19118.0,260641.000000
2001,19118.0,290717.416667
2002,19118.0,323259.250000
2003,19118.0,371517.916667
2004,19118.0,423675.916667
2005,19118.0,486571.500000
2006,19118.0,531186.250000
2007,19118.0,547926.500000
2008,19118.0,533627.250000
